<a href="https://colab.research.google.com/github/nid067/CODSOFT/blob/main/genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub as kh

# Download latest version
path = kh.dataset_download("hijest/genre-classification-dataset-imdb")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/genre-classification-dataset-imdb


In [2]:
import pandas as pd

In [3]:
import re

# Data Exploration using Pandas

In [4]:
train_data=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::',names=["movie_name","genre","summary"])
test_data=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt",sep=':::',names=["movie_name","summary"])
train_data.head()

/tmp/ipython-input-4-1852757236.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_data=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::',names=["movie_name","genre","summary"])
/tmp/ipython-input-4-1852757236.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_data=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt",sep=':::',names=["movie_name","summary"])


,movie_name,genre,summary
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [5]:
train_data.describe()

,movie_name,genre,summary
count,54214,54214,54214
unique,54214,27,54086
top,Nature's Fury: Storm of the Century (2006),drama,Grammy - music award of the American academy ...
freq,1,13613,12


# Data cleaning

In [6]:
def clean_text(text):
  text=text.lower()
  text=re.sub(r"[^a-z0-9\s]","",text)
  text=re.sub(R"\s+"," ",text).strip()
  return text

In [7]:
print(train_data.columns)

Index(['movie_name', 'genre', 'summary'], dtype='object')


In [11]:
#For training data
train_data['clean_plot']=train_data["summary"].apply(clean_text)
train_data[['movie_name', 'genre', 'summary','clean_plot']].head(10)

,movie_name,genre,summary,clean_plot
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,listening in to a conversation between his doc...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,a brother and sister with a past incestuous re...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,as the bus empties the students for their fiel...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,to help their unemployed father make ends meet...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,the films title refers not only to the unrecov...
6,Quality Control (2011),documentary,Quality Control consists of a series of 16mm ...,quality control consists of a series of 16mm s...
7,"""Pink Slip"" (2009)",comedy,In tough economic times Max and Joey have all...,in tough economic times max and joey have all ...
8,One Step Away (1985),crime,Ron Petrie (Keanu Reeves) is a troubled teen ...,ron petrie keanu reeves is a troubled teen who...
9,"""Desperate Hours"" (2016)",reality-tv,"A sudden calamitous event, causing great loss...",a sudden calamitous event causing great loss o...
10,Spirits (2014/I),horror,Four high school students embark on a terrify...,four high school students embark on a terrifyi...


In [16]:
# For testing data
test_data['clean_plot']=test_data['summary'].apply(clean_text)
test_data[['movie_name','summary','clean_plot']].head(10)

,movie_name,summary,clean_plot
1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar...",lr brane loves his life his car his apartment ...
2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch...",spain march 1964 quico is a very naughty child...
3,Off the Beaten Track (2010),One year in the life of Albin and his family ...,one year in the life of albin and his family o...
4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi...",his father has died he hasnt spoken with his b...
5,Er nu zhai (1955),Before he was known internationally as a mart...,before he was known internationally as a marti...
6,Riddle Room (2016),Emily Burns is being held captive in a room w...,emily burns is being held captive in a room wi...
7,L'amica (1969),The beautiful but neglected wife of a brillia...,the beautiful but neglected wife of a brillian...
8,Ina Mina Dika (1989),Vasu Inamdar (Ina) suffers from a disorder wh...,vasu inamdar ina suffers from a disorder where...
9,Equinox Special: Britain's Tornados (2005),An insight into the tornados that hit Kensal ...,an insight into the tornados that hit kensal r...
10,Press (2011),Press is a story of young people overwhelmed ...,press is a story of young people overwhelmed b...


Feature Extraction (Conversion of text to numbers in input data)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
obj=TfidfVectorizer(stop_words="english",max_features=5000)
X_train=obj.fit_transform(train_data['clean_plot'])

X_test=obj.transform(test_data['clean_plot'])